<a href="https://colab.research.google.com/github/ndrewwang/PyBaMM/blob/master/Week1_TheoreticalCapacities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Introduction to the Battery Fact Checker__
This is the test sheet for the Intercalation Station battery fact checker. 
- Battery Fact Checker
- Battery Auditor
- Battery Architecft
- Battery Designer



# Motivation
By understanding the principles and architecture behind batteries, we can build up an intuition that will help evaluate and identify future developments in the energy storage space. This curriculum is aimed at all those with a can-do technical attitude and enthusiasm in batteries. It will be particularly handy for analysts, reporters, investors, and students interested in joining Intercalation Station on the full journel of the creation of a simple tool in python. 

The inception of the tool was motivated by looking for a quick way to guesstimate answer questions such as:
- Why can silicon store so much more capacity than graphite?
- Why is LFP considered a lower energy material?
- Can we fact check this [Quantumscape investor slide](https://cdn.wccftech.com/wp-content/uploads/2020/10/Slide-2-740x416.jpg)?
- Why is lithium the preferred charge carrier?
- Why does Quantumscape's solid electrolyte have be a certain thickness?
- How much packaging efficiency is gained in Tesla's new 4680 cell?
- Why did JouleCP ask researchers to report volume of electrolyte used?

Initial goal will be to explain the mechanics of this [online calculator](http://lacey.se/apps/cell-energy/):  and build our own.


# __Week 1: Theoretical Capacities__
Lets define some units relevant to batteries:

#### _Capacity_
The amount of charge held by a battery. Where charge is the amount of current able to be maintained for a certain amount of time: $Q = It$. It typically written in units of mAh, for example your phone has a capacity of approx 3000 mAh. Roughly speaking, that 3000 mAh battery should be able to sustain a 3000 milli-amp (3 amp) current for 1 hour (3600 seconds). A good grasp of units will be useful going forward...

#### _Energy_
Has units of joules (J), the energy of a battery relates to the voltage of the battery: $\text{Energy} = VQ = VIt$. Voltage (V) multiplied with amps (A) gives power in the units watts (W). When this power is sustained for a certain time by a battery, the energy is given by watt-hours (Wh). Voltage and energy is equally important as capacity. An infinite capacity battery with a cell voltage lower than 3V would not even power your TV remote. 

#### _Power_
Power is the rate that energy is delivered, in watts (W). If energy was the amount of tea in a tea-pot, the power would be how quickly you could pour the tea from the spout of the pot. 

#### _Density_
We also care about how dense the battery is. This means how much capacity, energy, or power the battery can deliver relative to the size of the battery. Size can be defined either in terms of weight or volume. For example, the weight-relative capacity is sometimes refered to as the specific capacity or gravimetric capacity, and the volume-relative capacity is refered to as the volumetric capacity. 
- Specific capacity: mAh/g
- Volumetric capacity: mAh/L



# Faraday's Law
Michael Faraday discovered in the 1800s that the amount of a substance deposited electrochemically by passing some amount of change is directly related to the molecular mass of that substance. This remains the case for batteries where Li ions (or other metals) is shuttled between the cathode and anode materials.

This relationship is now known as Faraday's law, where the theoretical charge per mass or gravimetric capacity is given by:

<!-- $$ Q_\text{grav} = \frac{Q}{m} = \frac{nF}{M}$$ -->
$$ Q_\text{grav} =  \frac{nF}{M}\times\frac{1}{3600}\times 1000$$
where:
<!-- - $Q$ is the amount of charge (capacity)
- $m$ is the amount of mass -->
- $Q_\text{grav}$ is the gravimetric capacity [__mAh/g__]
- $n$ is the valence of the ion (1 for Li$^+$) 
- $M$ is the molecular mass of the material [__g__]
- $F$ is Faraday's constant $F = 96485$ [__C/mol__] representing the charge in one mole of electrons

If we work with molar masses in the [__g__] units, and want $Q_\text{grav}$ in the "industry standard" units of milli amp-hours per gram [__mAh/g__], we must scale the units:
- $\times\frac{1}{3600}$ scales the coulombs in amp-seconds [__A*s__] to amp-hours [__A*h__]
- $\times 1000$ scales the amp-hours [__A*h__] to milli amp-hours [__mAh__]



#Lets calculate some capacities, starting with lithium cobalt oxide! 
LiCoO2 follows this reaction during discharge, or lithium de-intercalation:

$$\text{LiCoO}_2 \rightarrow \text{CoO}_2 + \text{Li}^+ + e^-$$

Following these steps, the code below shows that LiCoO2 has a specific capacity of 273.8 mAh/g
1. What are the molar masses of each element?
2. What is the molar mass of the entire lithiated material?
3. Apply Faraday's Law to calculate the theoretical capacity:

In [1]:
# 1. What are the molar masses of each element?
#----------------------------------------------------------------
Li = 6.941 #g
Co = 58.933 #g
O = 15.999 #g

# 2. What is the molar mass of the entire lithiated material?
#----------------------------------------------------------------
LCO = Li + Co + O + O  #molecular mass of LiCoO2 (LCO)

# 3. Apply Faraday's Law to calculate the theoretical capacity:
#----------------------------------------------------------------
n = 1 #Lithium has a valence of +1
F = 96485 #C/mol

Q_grav = (n*F/LCO)*(1/3600)*1000 #Faradays law equation

#Displaying answer
print('LiCoO2 theoretical gravimetric capacity: \n')
print(str(round(Q_grav,2)) + ' mAh/g')

LiCoO2 theoretical gravimetric capacity: 

273.84 mAh/g


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Molar Mass Calculator

Lets make it easier to calculate the molar masses of a given electrode chemistry formula. You can run the block of code below to define a function __molmass.py__ that will take a string input and return the molar mass of the compound. 


In [2]:
# DEFINE MOLMASS FUNCTION - RUN AND MOVE ONTO THE NEXT CELL
%%writefile molmass.py
def molmass(molecule_string):
    import re
    from collections import Counter
    # Dictionary of molecular masses from periodic table
    molar_mass_table_dict = {'H': 1.00794, 'He': 4.002602, 'Li': 6.941, 'Be': 9.012182, 'B': 10.811, 'C': 12.0107, 'N': 14.0067,
                  'O': 15.9994, 'F': 18.9984032, 'Ne': 20.1797, 'Na': 22.98976928, 'Mg': 24.305, 'Al': 26.9815386,
                  'Si': 28.0855, 'P': 30.973762, 'S': 32.065, 'Cl': 35.453, 'Ar': 39.948, 'K': 39.0983, 'Ca': 40.078,
                  'Sc': 44.955912, 'Ti': 47.867, 'V': 50.9415, 'Cr': 51.9961, 'Mn': 54.938045,
                  'Fe': 55.845, 'Co': 58.933195, 'Ni': 58.6934, 'Cu': 63.546, 'Zn': 65.409, 'Ga': 69.723, 'Ge': 72.64,
                  'As': 74.9216, 'Se': 78.96, 'Br': 79.904, 'Kr': 83.798, 'Rb': 85.4678, 'Sr': 87.62, 'Y': 88.90585,
                  'Zr': 91.224, 'Nb': 92.90638, 'Mo': 95.94, 'Tc': 98.9063, 'Ru': 101.07, 'Rh': 102.9055, 'Pd': 106.42,
                  'Ag': 107.8682, 'Cd': 112.411, 'In': 114.818, 'Sn': 118.71, 'Sb': 121.760, 'Te': 127.6,
                  'I': 126.90447, 'Xe': 131.293, 'Cs': 132.9054519, 'Ba': 137.327, 'La': 138.90547, 'Ce': 140.116,
                  'Pr': 140.90465, 'Nd': 144.242, 'Pm': 146.9151, 'Sm': 150.36, 'Eu': 151.964, 'Gd': 157.25,
                  'Tb': 158.92535, 'Dy': 162.5, 'Ho': 164.93032, 'Er': 167.259, 'Tm': 168.93421, 'Yb': 173.04,
                  'Lu': 174.967, 'Hf': 178.49, 'Ta': 180.9479, 'W': 183.84, 'Re': 186.207, 'Os': 190.23, 'Ir': 192.217,
                  'Pt': 195.084, 'Au': 196.966569, 'Hg': 200.59, 'Tl': 204.3833, 'Pb': 207.2, 'Bi': 208.9804,
                  'Po': 208.9824, 'At': 209.9871, 'Rn': 222.0176, 'Fr': 223.0197, 'Ra': 226.0254, 'Ac': 227.0278,
                  'Th': 232.03806, 'Pa': 231.03588, 'U': 238.02891, 'Np': 237.0482, 'Pu': 244.0642, 'Am': 243.0614,
                  'Cm': 247.0703, 'Bk': 247.0703, 'Cf': 251.0796, 'Es': 252.0829, 'Fm': 257.0951, 'Md': 258.0951,
                  'No': 259.1009, 'Lr': 262, 'Rf': 267, 'Db': 268, 'Sg': 271, 'Bh': 270, 'Hs': 269, 'Mt': 278,
                  'Ds': 281, 'Rg': 281, 'Cn': 285, 'Nh': 284, 'Fl': 289, 'Mc': 289, 'Lv': 292, 'Ts': 294, 'Og': 294,
                  'ZERO': 0}

    #Parsing molecule formula 
    MATCH = {'(':')', '[':']', '{':'}'}
    tokens = re.findall(r"[A-Z][a-z]?|[A-Z]|[0-9].[0-9]+|[0-9]|[](){}[]", molecule)
    
    composition = Counter()
    match = []
    stack = []
   
    while tokens:
        # element with optional count
        if tokens[0].isalpha():
            element = tokens.pop(0)
            count = float(tokens.pop(0)) if tokens and tokens[0][0].isdigit() else 1
            composition[element] += count

        # start a sub-compound
        elif tokens[0] in ('(', '[', '{'):
            match.append(MATCH[tokens.pop(0)])
            stack.append(composition)
            composition = Counter()
            
        # matching close bracket ends a sub-compound with an optional count
        elif tokens[0] == match[-1]:
            tokens.pop(0)
            match.pop()
                
            repeat = float(tokens.pop(0)) if tokens and tokens[0][0].isdigit() else 1
            for element in composition.keys():
                composition[element] *= repeat
                
            composition.update(stack.pop())

        # "syntax" error in the formula
        else:
            if token[0] in (')', ']', '}'):
                raise ValueError(f"Error, mismatched bracket: "
                                 f"expected '{match[-1]}' got '{token[0]}'.")
            
            else:
                raise ValueError(f"Error, unrecognized token in "
                                 f"formula: '{token[0]}'.")

    # left over, unmatched brackets
    if match:
        brackets = ', '.join(f"'{b}'" for b in match[::-1])
        raise ValueError(f"Error, missing bracket(s): {brackets}.")
    
    parsed_dict = dict(composition)

    #Match parsed dictionary with number of elements to get total mass
    total_mass = 0
    for i in list(parsed_dict.keys()):
      current_mass = molar_mass_table_dict[i]*parsed_dict[i]
      total_mass = total_mass + current_mass

    return total_mass
execfile('molmass.py')

Writing molmass.py


# Extending to other chemistries
Now we have loaded the python function that takes chemical formulas and calculates the molar mass - we can give any electrode chemistry and calculate the theoretical capacity! 

Lets look at cathodes first by defining a python dictionary with LFP, LCO, NMC532, NMC811, and NCA. 